# ELI5 for interpretability
### Links!
- https://github.com/jwalluww/Machine-Learning-with-Python/blob/master/Bayesian%20Logistic%20Regression_bank%20marketing.ipynb
- https://towardsdatascience.com/interpretable-machine-learning-with-xgboost-9ec80d148d27
- https://blog.dominodatalab.com/shap-lime-python-libraries-part-1-great-explainers-pros-cons/

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

Banking dataset from github <br>
- label: "y", whether the customer subscribed to a long-term deposit

In [2]:
df1=pd.read_csv('https://raw.githubusercontent.com/madmashup/targeted-marketing-predictive-engine/master/banking.csv',header=0)

***
Let's have a look at the dataset

In [3]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp_var_rate    41188 non-null  float64
 16  cons_price_idx  41188 non-null  float64
 17  cons_conf_idx   41188 non-null 

In [4]:
df1.shape

(41188, 21)

In [5]:
df1.head()

,age,job,marital,education,default,housing,loan,contact,month,day_of_week,...,campaign,pdays,previous,poutcome,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,y
0,44,blue-collar,married,basic.4y,unknown,yes,no,cellular,aug,thu,...,1,999,0,nonexistent,1.4,93.444,-36.1,4.963,5228.1,0
1,53,technician,married,unknown,no,no,no,cellular,nov,fri,...,1,999,0,nonexistent,-0.1,93.200,-42.0,4.021,5195.8,0
2,28,management,single,university.degree,no,yes,no,cellular,jun,thu,...,3,6,2,success,-1.7,94.055,-39.8,0.729,4991.6,1
3,39,services,married,high.school,no,no,no,cellular,apr,fri,...,2,999,0,nonexistent,-1.8,93.075,-47.1,1.405,5099.1,0
4,55,retired,married,basic.4y,no,yes,no,cellular,aug,fri,...,1,3,1,success,-2.9,92.201,-31.4,0.869,5076.2,1


***

***
***
***
# ELI5

This is an imbalance dataset...

In [6]:
df1.y.value_counts()

0    36548
1     4640
Name: y, dtype: int64

Separate out Y and x

In [7]:
y = df1["y"].map({0:0, 1:1}) # Mostly commonly used to map yes to 1 and no to 0

X = df1.drop("y", axis=1)

In [8]:
X.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 41188 entries, 0 to 41187
Data columns (total 20 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   age             41188 non-null  int64  
 1   job             41188 non-null  object 
 2   marital         41188 non-null  object 
 3   education       41188 non-null  object 
 4   default         41188 non-null  object 
 5   housing         41188 non-null  object 
 6   loan            41188 non-null  object 
 7   contact         41188 non-null  object 
 8   month           41188 non-null  object 
 9   day_of_week     41188 non-null  object 
 10  duration        41188 non-null  int64  
 11  campaign        41188 non-null  int64  
 12  pdays           41188 non-null  int64  
 13  previous        41188 non-null  int64  
 14  poutcome        41188 non-null  object 
 15  emp_var_rate    41188 non-null  float64
 16  cons_price_idx  41188 non-null  float64
 17  cons_conf_idx   41188 non-null 

Target leakage...just take my word for it

In [9]:
X.drop("duration",inplace=True,axis=1)

In [10]:
X.dtypes

age                 int64
job                object
marital            object
education          object
default            object
housing            object
loan               object
contact            object
month              object
day_of_week        object
campaign            int64
pdays               int64
previous            int64
poutcome           object
emp_var_rate      float64
cons_price_idx    float64
cons_conf_idx     float64
euribor3m         float64
nr_employed       float64
dtype: object

## Create pipeline

In [11]:
from sklearn.compose import ColumnTransformer # deal with all variable types in one block
from sklearn.preprocessing import OneHotEncoder # Look at me, I'm a data scientist!
from sklearn.pipeline import Pipeline # Run the whole model process in one block
from sklearn.model_selection import train_test_split, GridSearchCV # splitting data and finding ideal hyperparameters
from sklearn.metrics import accuracy_score, classification_report

# Our algorithms, by from the easiest to the hardest to intepret.
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from xgboost.sklearn import XGBClassifier

#### Start with binarizing categorical variables and then run the model

Split features, numerical and categorical

In [12]:
num_features = ["age", "campaign", "pdays", "previous", "emp_var_rate", 
                "cons_price_idx", "cons_conf_idx","euribor3m", "nr_employed"]

cat_features = ["job", "marital", "education","default", "housing", "loan",
                "contact", "month", "day_of_week", "poutcome"]

Column transformer handles numerical and categorical all at once

In [13]:
# Transform numerical features by just passing them through, then provide list
# Transform categorical features by one hot encoder, then provide list
preprocessor = ColumnTransformer([("numerical","passthrough",num_features),
                                  ("categorical",OneHotEncoder(sparse=False,handle_unknown="ignore"),cat_features)])

Use a pipeline below to connect preprocessory to model

In [14]:
# Logistic Regression
lr_model = Pipeline([("preprocessor", preprocessor), 
                     ("model", LogisticRegression(class_weight="balanced", solver="liblinear", random_state=42))])

# Decision Tree
dt_model = Pipeline([("preprocessor", preprocessor), 
                     ("model", DecisionTreeClassifier(class_weight="balanced"))])

# Random Forest
rf_model = Pipeline([("preprocessor", preprocessor), 
                     ("model", RandomForestClassifier(class_weight="balanced", n_estimators=100, n_jobs=-1))])

# XGBoost
xgb_model = Pipeline([("preprocessor", preprocessor), 
                      # Add a scale_pos_weight to make it balanced
                      ("model", XGBClassifier(scale_pos_weight=(1 - y.mean()), n_jobs=-1))])

Split data below

In [15]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, test_size=0.3, random_state=42)

## ELI5 to interpret "white box" models

#### Logistic Regression

Grid search hypterparameters

In [16]:
# lr_model - the model in question
# model__C - 
# n_jobs - "-1" runs your computations in parallel
# cv - cross validation, so we're doing 5-fold cross validation
# scoring - accuracy is chosen here (True Pos + True Neg / Total)
gs = GridSearchCV(lr_model, {"model__C": [1, 1.3, 1.5]}, n_jobs=-1, cv=5, scoring="accuracy")
gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('numerical',
                                                                         'passthrough',
                                                                         ['age',
                                                                          'campaign',
                                                                          'pdays',
                                                                          'previous',
                                          

View parameters and score

In [17]:
print(gs.best_params_)
print(gs.best_score_)

{'model__C': 1.5}
0.8294200662433966


Use the best parameters for the model

In [18]:
lr_model.set_params(**gs.best_params_)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical', 'passthrough',
                                                  ['age', 'campaign', 'pdays',
                                                   'previous', 'emp_var_rate',
                                                   'cons_price_idx',
                                                   'cons_conf_idx', 'euribor3m',
                                                   'nr_employed']),
                                                 ('categorical',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dt...
              

Fit model on entire training set

In [19]:
lr_model.fit(X_train,y_train)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical', 'passthrough',
                                                  ['age', 'campaign', 'pdays',
                                                   'previous', 'emp_var_rate',
                                                   'cons_price_idx',
                                                   'cons_conf_idx', 'euribor3m',
                                                   'nr_employed']),
                                                 ('categorical',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dt...
              

Generate predictions on the test set

In [20]:
y_pred=lr_model.predict(X_test)

Model Accuracy

In [21]:
accuracy_score(y_test,y_pred)

0.8277899166464352

Full classification

In [22]:
print(classification_report(y_test,y_pred))

              precision    recall  f1-score   support

           0       0.95      0.85      0.90     10965
           1       0.35      0.62      0.45      1392

    accuracy                           0.83     12357
   macro avg       0.65      0.74      0.67     12357
weighted avg       0.88      0.83      0.85     12357



### ELI5 to visualize weights associated with each feature

In [23]:
import eli5

C:\Users\wallj\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
C:\Users\wallj\Anaconda3\lib\site-packages\sklearn\utils\deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)


Weights associated with each feature. But there is not feature name...

In [24]:
eli5.show_weights(lr_model.named_steps["model"])

Weight?,Feature
+1.020,x49
+0.721,x7
+0.617,x5
+0.510,x61
+0.321,x14
+0.272,x17
+0.271,x45
+0.263,x42
+0.243,x47
+0.139,x58


Data gymnastics to show feature names

Preprocess to get final variable names before modeling

In [27]:
preprocessor=lr_model.named_steps["preprocessor"]

Process categorical features and gather them

In [30]:
ohe_categories=preprocessor.named_transformers_["categorical"].categories_

Get final list of categorical variable names

In [32]:
new_ohe_features = [f"{col}__{val}" for col, vals in zip(cat_features, ohe_categories) for val in vals]

Numerical features and new list of categorical features

In [34]:
all_features = num_features + new_ohe_features

Quick peak at dataframe

In [35]:
pd.DataFrame(lr_model.named_steps["preprocessor"].transform(X_train), columns=all_features).head()

,age,campaign,pdays,previous,emp_var_rate,cons_price_idx,cons_conf_idx,euribor3m,nr_employed,job__admin.,...,month__oct,month__sep,day_of_week__fri,day_of_week__mon,day_of_week__thu,day_of_week__tue,day_of_week__wed,poutcome__failure,poutcome__nonexistent,poutcome__success
0,26.0,1.0,999.0,0.0,1.1,93.994,-36.4,4.856,5191.0,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
1,59.0,1.0,999.0,0.0,-2.9,92.201,-31.4,0.884,5076.2,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0,0.0
2,28.0,6.0,999.0,0.0,1.4,93.918,-42.7,4.961,5228.1,0.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,1.0,0.0
3,47.0,18.0,999.0,0.0,1.4,93.918,-42.7,4.968,5228.1,0.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1.0,0.0
4,36.0,1.0,999.0,0.0,1.4,93.918,-42.7,4.960,5228.1,0.0,...,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0


Let's see some feature names!

In [36]:
eli5.show_weights(lr_model.named_steps["model"], feature_names=all_features)

Weight?,Feature
+1.020,month__mar
+0.721,euribor3m
+0.617,cons_price_idx
+0.510,poutcome__success
+0.321,job__retired
+0.272,job__student
+0.271,month__aug
+0.263,contact__cellular
+0.243,month__jul
+0.139,day_of_week__wed


#### Use ELI5 to explain a specific prediction

Customer number 4's input information

In [39]:
i=4
X_test.iloc[i]

age                        48
job                technician
marital               married
education            basic.9y
default                    no
housing                    no
loan                       no
contact              cellular
month                     may
day_of_week               thu
campaign                    3
pdays                     999
previous                    0
poutcome          nonexistent
emp_var_rate             -1.8
cons_price_idx         92.893
cons_conf_idx           -46.2
euribor3m               1.327
nr_employed            5099.1
Name: 35534, dtype: object

They ended up subscribing after the campaign

In [40]:
y_test.iloc[i]

1

Looking at cusomter #4 again below, we can see that nr_employed played a big part in the subscription as well as consumer price indeex.

In [41]:
eli5.show_prediction(lr_model.named_steps["model"],
                     lr_model.named_steps["preprocessor"].transform(X_test)[i],
                     feature_names=all_features, show_feature_values=True)

#### Decision Tree

In [42]:
gs = GridSearchCV(dt_model, {"model__max_depth": [3, 5, 7], 
                             "model__min_samples_split": [2, 5]}, 
                  n_jobs=-1, cv=5, scoring="accuracy")

gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('numerical',
                                                                         'passthrough',
                                                                         ['age',
                                                                          'campaign',
                                                                          'pdays',
                                                                          'previous',
                                          

Best parameters and score again.

Depth of 3, min sample split of 2, 0.85 score

In [43]:
print(gs.best_params_)
print(gs.best_score_)

{'model__max_depth': 3, 'model__min_samples_split': 2}
0.8461033421765723


Run the model with the best parameters

In [45]:
dt_model.set_params(**gs.best_params_)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical', 'passthrough',
                                                  ['age', 'campaign', 'pdays',
                                                   'previous', 'emp_var_rate',
                                                   'cons_price_idx',
                                                   'cons_conf_idx', 'euribor3m',
                                                   'nr_employed']),
                                                 ('categorical',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dt...
              

Fit the model on the training data and find predictions

In [46]:
dt_model.fit(X_train, y_train)
y_pred = dt_model.predict(X_test)

Accuracy

In [47]:
accuracy_score(y_test, y_pred)

0.8438132232742575

Full Classification table

In [48]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.94      0.88      0.91     10965
           1       0.37      0.57      0.45      1392

    accuracy                           0.84     12357
   macro avg       0.66      0.72      0.68     12357
weighted avg       0.88      0.84      0.86     12357



For decision trees, ELI5 gives feature importance, but not feature direction.

In [49]:
eli5.show_weights(dt_model.named_steps["model"],feature_names=all_features)

Weight,Feature
0.7766,nr_employed
0.1494,cons_conf_idx
0.0368,cons_price_idx
0.0160,poutcome__success
0.0149,job__blue-collar
0.0058,contact__telephone
0.0005,age
0,day_of_week__fri
0,job__services
0,job__student


Explanation for a single prediction is calculated by following the decision path in the tree! How cool! Adding up contribution from each node crossed into the overall probability predicted.

In [50]:
eli5.show_prediction(dt_model.named_steps["model"], 
                     dt_model.named_steps["preprocessor"].transform(X_test)[i],
                     feature_names=all_features, show_feature_values=True)

#### Random Forest

In [51]:
gs = GridSearchCV(rf_model, {"model__max_depth": [10, 15], 
                             "model__min_samples_split": [5, 10]}, 
                  n_jobs=-1, cv=5, scoring="accuracy")

gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('numerical',
                                                                         'passthrough',
                                                                         ['age',
                                                                          'campaign',
                                                                          'pdays',
                                                                          'previous',
                                          

Best parameters and score

In [52]:
print(gs.best_params_)
print(gs.best_score_)

{'model__max_depth': 15, 'model__min_samples_split': 5}
0.8775971218062798


In [456]:
rf_model.set_params(**gs.best_params_)

Pipeline(memory=None,
         steps=[('preprocessor',
                 ColumnTransformer(n_jobs=None, remainder='drop',
                                   sparse_threshold=0.3,
                                   transformer_weights=None,
                                   transformers=[('numerical', 'passthrough',
                                                  ['age', 'campaign', 'pdays',
                                                   'previous', 'emp_var_rate',
                                                   'cons_price_idx',
                                                   'cons_conf_idx', 'euribor3m',
                                                   'nr_employed']),
                                                 ('categorical',
                                                  OneHotEncoder(categories='auto',
                                                                drop=None,
                                                                dt...
              

Train the model and find predictions

In [53]:
rf_model.fit(X_train, y_train)
y_pred = rf_model.predict(X_test)

Accuracy

In [54]:
accuracy_score(y_test, y_pred)

0.89277332685927

Full classification report

In [55]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.97      0.94     10965
           1       0.55      0.28      0.37      1392

    accuracy                           0.89     12357
   macro avg       0.73      0.63      0.66     12357
weighted avg       0.87      0.89      0.88     12357



Feature importance plus standard deviations!

In [56]:
eli5.show_weights(rf_model.named_steps["model"], 
                  feature_names=all_features)

Weight,Feature
0.1322 ± 0.0211,age
0.1291 ± 0.1505,euribor3m
0.0751 ± 0.0094,campaign
0.0568 ± 0.1447,emp_var_rate
0.0552 ± 0.1445,nr_employed
0.0330 ± 0.0639,cons_conf_idx
0.0241 ± 0.0456,cons_price_idx
0.0192 ± 0.0066,housing__no
0.0190 ± 0.0068,housing__yes
0.0171 ± 0.0066,marital__married


#### XGBoost, Long may she reign

In [57]:
gs = GridSearchCV(xgb_model, {"model__max_depth": [5, 10],
                              "model__min_child_weight": [5, 10],
                              "model__n_estimators": [25]},
                  n_jobs=-1, cv=5, scoring="accuracy")

gs.fit(X_train, y_train)

GridSearchCV(cv=5, error_score=nan,
             estimator=Pipeline(memory=None,
                                steps=[('preprocessor',
                                        ColumnTransformer(n_jobs=None,
                                                          remainder='drop',
                                                          sparse_threshold=0.3,
                                                          transformer_weights=None,
                                                          transformers=[('numerical',
                                                                         'passthrough',
                                                                         ['age',
                                                                          'campaign',
                                                                          'pdays',
                                                                          'previous',
                                          

Best parameters, score, set best parameters, train the model, find predictions

In [59]:
print(gs.best_params_)
print(gs.best_score_)
xgb_model.set_params(**gs.best_params_)
xgb_model.fit(X_train, y_train)
y_pred = xgb_model.predict(X_test)

{'model__max_depth': 5, 'model__min_child_weight': 5, 'model__n_estimators': 25}
0.9004890997440735


Accuracy

In [60]:
accuracy_score(y_test, y_pred)

0.90102775754633

Full classification report

In [61]:
print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

           0       0.91      0.99      0.95     10965
           1       0.70      0.22      0.33      1392

    accuracy                           0.90     12357
   macro avg       0.80      0.60      0.64     12357
weighted avg       0.88      0.90      0.88     12357



Weights - still using the feature names we obtained earlier.

In [62]:
eli5.show_weights(xgb_model.named_steps["model"], 
                  feature_names=all_features)

Weight,Feature
0.4710,nr_employed
0.1112,poutcome__success
0.0578,cons_conf_idx
0.0349,pdays
0.0316,month__oct
0.0298,euribor3m
0.0217,job__blue-collar
0.0181,previous
0.0173,poutcome__failure
0.0172,contact__cellular
